In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from math import floor
from math import ceil
import csv
from statistics import stdev
import numpy as np

In [ ]:
RESULTS_DIR = "/home/local/ASUAD/kjatkins/plotting"

In [ ]:
def get_data(csv_file, key, filter_func = lambda x: True):
    print(csv_file)
    x = []
    y = []
    csv_reader = csv.DictReader(open(csv_file, "r"), delimiter=";")
    for row in csv_reader:
        _x = int(row["Interaction"])
        if filter_func(_x):
            x.append(_x)
            y.append(float(row[key]))
            
    return x, y

def plot(data, ylabel, title, output_filepath, xlabel="Interaction #"):
    
    fig, ax = plt.subplots()
    
    for label in data:
        
        x, y, = data[label]["x"], data[label]["y"]
        ax.plot(x, y, label=label)
    
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.legend()
    
    fig.savefig(output_filepath)
    pass

In [ ]:

KEYS = {
    
    "Success Rate": "Success Rate",
    "Variational Difference (Samples)": "Variational Difference",
    "Elapsed Time": "Elapsed Time"
    
}
    

In [ ]:

domains = ['tireworld_og']
domainName = {'tireworld_og':'Original Tireworld'}
dataList = []
for domain in domains:
    ipml_r_filepath = "%s/ipml_randomized_%s.csv" % (RESULTS_DIR, domain)
    ipml_s_filepath = "%s/ipml_sequential_%s.csv" % (RESULTS_DIR, domain)
    glib_g1_filepath = "%s/glib_%s_glib_g1_lndr.csv" % (RESULTS_DIR, domain)
    glib_l2_filepath = "%s/glib_%s_glib_l2_lndr.csv" % (RESULTS_DIR, domain)

    for key in KEYS:

        data = {}

        fileprefix = KEYS[key][0]
        output_filepath = "%s/%s_%s.png" % (RESULTS_DIR, domain, fileprefix)

        x, y = get_data(ipml_r_filepath, key)
        filter_func = lambda y: y <= max(x)

        data["IPML Randomized (Ours)"] = {
            "x": x,
            "y": y,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        x, y = get_data(ipml_s_filepath, key)
        data["IPML Sequential (Ours)"] = {
            "x": x,
            "y": y,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        x, y = get_data(glib_g1_filepath, key)
        data["GLIB (G1)"] = {
            "x": x,
            "y": y,
            "key": KEYS[key],
            'domain': domainName[domain]
        }

        x, y = get_data(glib_l2_filepath, key)
        data["GLIB (L2)"] = {
            "x": x,
            "y": y,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        dataList.append(data)
    
    


In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(20, 10))
ncols = 4
nrows=3

spec2 = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure=fig2)


f2_axs = []

for i in range(0, nrows):
    data = dataList[i]
    for j in range(0, ncols):

        x,y,key,domain = data['IPML Randomized (Ours)']['x'], data['IPML Randomized (Ours)']['y'],data['IPML Randomized (Ours)']['key'], data['IPML Randomized (Ours)']['domain']

        f2_ax = fig2.add_subplot(spec2[i, j])
        f2_axs.append(f2_ax)
        
        if x[len(x)-1] != 400:
            x.append(400)
            y.append(y[len(y)-1])
        
        y = np.array(y)
        
        
        x4,y4,key4 = data['IPML Sequential (Ours)']['x'], data['IPML Sequential (Ours)']['y'],data['IPML Sequential (Ours)']['key']
        
        if x4[len(x4)-1] != 400:
            x4.append(400)
            y4.append(y4[len(y4)-1])
        
        y4=np.array(y4)
        
        
        f2_ax.plot(x4,y4,label='IPML Sequential (Ours)')

        f2_ax.plot(x,y,label='IPML Randomized (Ours)')
        x2,y2,key2 = data['GLIB (G1)']['x'], data['GLIB (G1)']['y'], data['GLIB (G1)']['key']
        y2 = np.array(y2)
        f2_ax.plot(x2,y2,label='GLIB (G1)')
        x3,y3,key3 = data['GLIB (L2)']['x'], data['GLIB (L2)']['y'], data['GLIB (L2)']['key']
        y3 = np.array(y3)
        f2_ax.plot(x3,y3,label='GLIB (L2)')
        
        
        
        yStdev = stdev(y)
        y2Stdev = stdev(y2)
        y3Stdev = stdev(y3)
        y4Stdev = stdev(y4)
        if i == 0:
            f2_ax.set_title(domain, fontsize=20)
        

        if key == "Success Rate":
            f2_ax.set_ylim(-0.05,1.05)
        elif key == "Elapsed Time":
            f2_ax.set_ylim(-25,6e2)
        else:
            f2_ax.set_ylim(-0.05, 0.9)

        
        if j == 0:
            f2_ax.set_ylabel(key, fontsize=19)
            
        else:
            f2_ax.set_yticklabels([])
        
        if i < nrows-1:
            f2_ax.set_xticklabels([])
        
        f2_ax.tick_params(axis='both', which='major', labelsize=18)
        

        
        f2_ax.fill_between(x, y-yStdev, y+yStdev, alpha=0.2)
        f2_ax.fill_between(x2,y2-y2Stdev, y2+y2Stdev, alpha=0.2)
        f2_ax.fill_between(x3,y3-y3Stdev, y3+y3Stdev, alpha=0.2)
        f2_ax.fill_between(x4,y4-y4Stdev,y4+y4Stdev,alpha=0.2)
        
        

fig2.legend(loc='upper center',labels=['IPML Randomized (Ours)', 'IPML Sequential (Ours)', 'GLIB (G1)', 'GLIB (L2)'], prop={'size': 18}, bbox_to_anchor=(0.5, 1.13), ncol=4, frameon=False, fancybox=False,borderpad=0)

fig2.text(0.53, -0.03, 'Interaction #', ha='center', fontsize=18)




plt.savefig(fname='plot_false.png',tight_layout=False, bbox_inches='tight')
